In [132]:
import numpy as np
import urllib.request
import re
from itertools import permutations, product
from collections import namedtuple
from heapq import heappop, heappush

# It's day 22
day = 22

# Load file from url
txt = urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone/Advent-of-Code/master/2016/input_files/input{day}.txt').read().decode('utf-8')
lines = txt.split('\n')[:-1]

In [133]:
def parse_int(line):
  return [int(x) for x in re.findall(r'\d+', line)]

def viable_pair(a, b):
  return a != b and 0 < a.used <= b.avail

Node = namedtuple('Node', 'x, y, size, used, avail, perc')

In [134]:
# PART ONE
grid = [Node(*parse_int(line)) for line in lines if line.startswith('/dev')]
count = sum([viable_pair(a, b) for a, b in permutations(grid, 2)])
print(f'Viable pairs of my input are {count}')

Viable pairs of my input are 1034


In [135]:
# GUESS WHO'S BACK
State = namedtuple('State', 'goalpos, emptypos')

def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def heuristic(state):
  # Our heuristic is the city-distance between datagoal position of this state and (0,0)
  return sum(state.goalpos)

def moves(state):
  # Possible moves from this state
  (x, y), (x_empty, y_empty) = state
  for x2, y2 in list(product([x_empty], [y_empty-1, y_empty+1])) + list(product([x_empty-1, x_empty+1], [y_empty])):
    if 0 <= x2 <= x_max and 0 <= y2 <= y_max and legal_move(grid[x2, y2], grid[empty]):
      newgoal = (x_empty, y_empty) if x2 == x and y2 == y else (x, y)
      yield State(newgoal, (x2, y2))

def emptypos(grid):
  for node in grid.items():
    if node[1].used == 0:
      return node[1].x, node[1].y

def legal_move(a, b):
  return True if a.used <= b.size else False

def step(prev, state):
  return 0 if state is None else 1 + step(prev, prev[state])

In [137]:
# We restructure the grid to ease notation to problem solver
grid = {(node.x, node.y): node for node in grid}
x_max = max(node[1].x for node in grid.items())
y_max = max(node[1].y for node in grid.items())
empty = emptypos(grid)
ret = problem_solver(State((x_max, 0), empty), heuristic, moves)
print(f'Fewest number of steps required is {ret - 1}')

Fewest number of steps required is 261
